In [2]:
import pickle
import numpy as np
from time import time
import os
from helpers import TextVectorizer
from model import MyModel

In [3]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
import ipywidgets as widgets
from IPython.display import display

In [5]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

In [6]:
print("Tensorflow version: ", tf.__version__)

Tensorflow version:  2.0.0-alpha0


In [7]:
with open('data/data_packed.pickle', 'rb') as f:
    data_packed = pickle.load(f)

category_map = data_packed['category_map']
characters = data_packed['characters']
data = data_packed['downloaded']
categories = sorted(data.keys())

In [8]:
print(f"Categroies: {len(categories)}")
print(f"Characters: {len(characters)}")

Categroies: 124
Characters: 104


In [9]:
vectorizer = TextVectorizer(characters, categories)

In [10]:
SEQUENCE_SIZE = 256
BATCH_SIZE = 64

In [11]:
data['ahamkara']['cards']

{'Ghost Fragment: Legends 3': 'After great deliberation it was determined that the Ahamkara be made extinct. \nIt was not an easy decision. Power had been obtained from the bargains, and the City needed power. Knowledge had been gleaned, and the Ahamkara knew answers to questions no one had known to ask.\nBut the price was too high. And no edict or forbearance seemed to stop Guardians from seeking them out, driven by hope, or vengeance, or despair.\nThe call had to be silenced. So the Great Hunt did its work.\nAnd thus the Ahamkara were made extinct, their call silenced, their solipsistic flatteries erased, their great design - if it ever existed - broken.\nOf this you can be assured, oh reader mine.␃',
 'Ghost Fragment: Warlock': "Why did I set her on the trail?\nYou try and try and try to explain, but no one ever understands. No one who's not a Warlock. Who hasn't spent a dozen years scouring the ruins for one string of symbols, one clean code, one black talon. Titans just make a hmp

In [12]:
dataset = []
for category in categories:
    for card in data[category]['cards'].values():
        item = vectorizer.text_to_array(text=card, category=category)
        dataset.append(item)
    for entry in data[category]['entries'].values():
        item = vectorizer.text_to_array(text=entry, category=category)
        dataset.append(item)

In [13]:
def embed_data(data):
    """
    Equalizes the length of each sequence is the given data
    by appending "␃" at the end of shorter entries
    """
    ans = []
    max_len = max([len(i) for i in data])
    for point in data:
        ans.append(vectorizer.repeat_last(point, target_len=max_len))
    return np.array(ans)

In [14]:
vectorizer.array_to_text(embed_data(dataset[:10])[6])

"The Leviathan's Warning\n++We live on the edge of a war-\n-a war between Formless and Form++\n++between the Deep and the Sky-\n++MY EYES ARE WIDE, MY GAZE IS LONG++\n-Across the universe, as far as I see++\n++the Sky works to charge its fires- \n -and the Deep drowns the ash++\n-Sky builds gentle places, safe for life++\n++Beloved Fundament, refuge of trillions-\n-The Sky treasures this rich place++\n-BUT THE DEEP IS HERE WITH US-\n++Cold logic tests our walls-\n-The Deep claims its dominion++\n++A ruthless, final age -\nAurash's Protest\nOld Leviathan, creature of myth, this world is no refuge. We live short, hard lives. We die in the dark. The storm above us will never end. And soon the God-Wave will take us all. Above us there are only stormjoys, monsters, and moons of apocalypse. Let us go down, down, where we may discover truth, some power to avenge ourselves upon our betrayers, some hope of survival.\nThe Leviathan's Hope\n-What power calls you++\n++Down to the deep?-\n++What in

In [15]:
LSTM1_SIZE = 512
LSTM2_SIZE = 1024
model = MyModel(LSTM1_units=LSTM1_SIZE,
                LSTM2_units=LSTM2_SIZE,
                output_size=len(characters))

W0420 18:16:55.023319  8768 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x00000150E5267A58>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0420 18:16:55.030318  8768 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x00000150E528A1D0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [16]:
def get_batch(data):
    batches = len(data) // BATCH_SIZE
    for i in range(batches):
        inputs = []
        targets = []
        batch = data[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        embedded = embed_data(batch)
        for single_batch in embedded:
            inputs.append(single_batch[:-1])
            targets.append(single_batch[1:,:len(characters)])
        yield np.array(inputs), np.array(targets)

In [17]:
def split_batch_to_sequences(inputs, targets):
    """
    Splits batches into sequences with equal length
    NOTE:
    Sequences length will be close but not always equal
    to SEQUENCE_SIZE
    """
    sequences = np.ceil(len(inputs[0]) / SEQUENCE_SIZE)
    inputs_split = np.array_split(inputs, sequences, axis=1)
    targets_split = np.array_split(targets, sequences, axis=1)
    for input_seq, target_seq in zip(inputs_split, targets_split):
        yield input_seq, target_seq

In [18]:
x,y = next(get_batch(dataset))

In [19]:
print(f"x: {x.shape}")
print(f"y: {y.shape}")

x: (64, 3595, 228)
y: (64, 3595, 104)


In [20]:
x_seq, y_seq = next(split_batch_to_sequences(x,y))

In [21]:
print(f"x_seq: {x_seq.shape}")
print(f"y_seq: {y_seq.shape}")

x_seq: (64, 240, 228)
y_seq: (64, 240, 104)


In [22]:
print(f"Input:\t{repr(vectorizer.array_to_text(x_seq[0]))}")
print(f"Target:\t{repr(vectorizer.array_to_text(y_seq[0]))}\n")

Input:	'After great deliberation it was determined that the Ahamkara be made extinct. \nIt was not an easy decision. Power had been obtained from the bargains, and the City needed power. Knowledge had been gleaned, and the Ahamkara knew answers to q'
Target:	'fter great deliberation it was determined that the Ahamkara be made extinct. \nIt was not an easy decision. Power had been obtained from the bargains, and the City needed power. Knowledge had been gleaned, and the Ahamkara knew answers to qu'



In [31]:
# model needs to run on some data before it can be summarized
vectorizer.array_to_text(model(x_seq)[0].numpy())

'wêww999rrrrrrrrrr99999999}}}}9999999i99}}}}}}}tttii999999ttttwwt9ttttttttt}}iiiiiiii9999wwwwwwwwwwwtt}i999999}999999}}}}ttt}tttttttttt}}}}}}9}ttttttww99wwtttt}}}99}}}ttiiii}}i}}}}}}}}wwww999tttttttttttttttttttttttt99999ttwwwwwwwwwwwwwww9iii'

In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm (UnifiedLSTM)   multiple                  1517568   
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) multiple                  6295552   
_________________________________________________________________
dense (Dense)                multiple                  106600    
Total params: 7,919,720
Trainable params: 7,919,720
Non-trainable params: 0
_________________________________________________________________


In [26]:
if not os.path.exists("checkpoints/"):
    os.makedirs("checkpoints/")

In [27]:
OPTIMIZER = tf.optimizers.Nadam(learning_rate=0.001, beta_2=0.9999, epsilon=0.1)
LOSS = tf.losses.categorical_crossentropy
EPOCHS = 50
DROPOUT_RATE = 0.3
CHECKPOINT_FORMAT = "checkpoints/Nadam{epoch}.ckpt" 

In [28]:
def single_train_step(inputs, targets):
    with tf.device("/device:GPU:0"):
        with tf.GradientTape() as tape:
            preds = model(inputs=inputs, remember=True, dropout_rate=DROPOUT_RATE)
            loss = LOSS(targets, preds)
        grads = tape.gradient(loss, model.trainable_variables)
        OPTIMIZER.apply_gradients(zip(grads, model.trainable_variables))
        return loss

In [103]:
history = []
for epoch in range(1, EPOCHS+1):
    np.random.shuffle(dataset)
    epoch_start = time()
    
    losses = []
    num_batches = len(dataset) // BATCH_SIZE

    for batch_num, (inputs, targets) in enumerate(get_batch(dataset), 1):
        model.forget()  # reset states
        batch_start = time()
        num_sequences = int(np.ceil(len(inputs[0]) / SEQUENCE_SIZE))
        for (sequence_num,
            (input_sequence,
            target_sequence)) in enumerate(split_batch_to_sequences(inputs, targets), 1):
    
            loss = single_train_step(input_sequence, target_sequence)
            print(f"Epoch {epoch}/{EPOCHS}\t"
                  f"Batch {batch_num}/{num_batches}\t"
                  f"Sequence {sequence_num}/{num_sequences}\t"
                  f"Loss {loss.numpy().mean():.4f}", end='\r')
            losses.append(loss.numpy().mean())

    print(f"Epoch {epoch} finished!\t"
          f"Time: {time()-epoch_start:.0f}s per epoch\t"
          f"Average loss: {np.mean(losses):.4f}")
    history.append(losses)
    model.save_weights(CHECKPOINT_FORMAT.format(epoch=epoch))

Epoch 1 finished!	Time: 714s per epoch	Average loss: 1.2793
Epoch 2 finished!	Time: 719s per epoch	Average loss: 0.8622
Epoch 3 finished!	Time: 664s per epoch	Average loss: 0.7301
Epoch 4 finished!	Time: 690s per epoch	Average loss: 0.6052
Epoch 5 finished!	Time: 687s per epoch	Average loss: 0.5540
Epoch 6 finished!	Time: 686s per epoch	Average loss: 0.5116
Epoch 7 finished!	Time: 665s per epoch	Average loss: 0.4928
Epoch 8 finished!	Time: 691s per epoch	Average loss: 0.4440
Epoch 9 finished!	Time: 654s per epoch	Average loss: 0.4447
Epoch 10 finished!	Time: 681s per epoch	Average loss: 0.4018
Epoch 11 finished!	Time: 659s per epoch	Average loss: 0.3949
Epoch 12 finished!	Time: 660s per epoch	Average loss: 0.3880
Epoch 13 finished!	Time: 674s per epoch	Average loss: 0.3669
Epoch 14 finished!	Time: 676s per epoch	Average loss: 0.3501
Epoch 15 finished!	Time: 672s per epoch	Average loss: 0.3457
Epoch 16 finished!	Time: 673s per epoch	Average loss: 0.3353
Epoch 17 finished!	Time: 655s per

In [104]:
fig = figure()

history_avg = [np.mean(i) for i in history]
X = range(len(history_avg))
Y = history_avg

fig.line(X,Y, legend="Loss", color="blue")

show(fig)